In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import h5py
import numpy as np
from os import getcwd as cwd
from os.path import join as pj
import torch
import torch.nn.functional as F
import torch.utils.data as data

# Dataset
from dataset.classification.dataset import insects_dataset
# Model
from model.resnet.resnet import ResNet

# Test Config

In [3]:
class args:
    # experiment_name
    experiment_name = "resnet50_b20_r45_lr1e-5_crossvalid_20200806_All"
    # paths
    all_data_path = pj(cwd(), "data/all_classification_data/classify_insect_std_20200806")
    model_root = pj(cwd(), "output_model/classification/ResNet101", experiment_name)
    knowledge_path = pj(cwd(), "data/insect_knowledge", experiment_name)
    # train config
    model_name = "resnet50" # choice ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152"]
    bs = 20
    pretrain = True
    param_freeze = False
    activation_function = "ReLU" # choice ["ReLU", "LeakyReLU", "RReLU"]
    decoder = None # choice [None, "Concatenate", "FPN"]
    add_linear = False

In [4]:
dataset_name = args.all_data_path.split('/')[-1]
if dataset_name == 'classify_insect_std':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']
elif dataset_name == 'classify_insect_std_resizeFAR':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']
elif dataset_name == 'classify_insect_std_resize':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']
elif dataset_name == 'classify_insect_std_plus_other':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera', 'Other']
elif dataset_name == 'classify_insect_std_20200806':
    args.labels = ['Diptera', 'Ephemeridae', 'Ephemeroptera', 
                   'Lepidoptera', 'Plecoptera', 'Trichoptera']

### generate insect knowledge
- knowledge = softmax distribution

In [5]:
model_save_path = pj(args.model_root, "final.pth")
model = ResNet(args.model_name, len(args.labels), pretrain=args.pretrain, param_freeze=args.param_freeze, activation_function=args.activation_function, decoder=args.decoder, add_linear=args.add_linear).cuda()
model.load_state_dict(torch.load(model_save_path))

activation_function == ReLU


<All keys matched successfully>

In [6]:
if os.path.exists(args.knowledge_path) is False:
    os.makedirs(args.knowledge_path)

In [7]:
with h5py.File(args.all_data_path) as f:
    X = f["X"][:]
    Y = f["Y"][:]

all_dataset = insects_dataset(X, Y, training=False)
all_dataloader = data.DataLoader(all_dataset, args.bs, num_workers=args.bs, shuffle=False)

In [8]:
def generate_insect_knowledge(model, test_dataloader):
    """
        generate softmax distribution of insect recognition
        Args:
            - model: pytorch model
            - test_dataloader: torchvision dataloader
    """
    model.eval()
    insect_knowledges = []
    for x in test_dataloader:
        x = x.cuda()
        out = model(x)
        out = F.softmax(out, dim=-1)
        insect_knowledges.extend(list(out.cpu().detach().numpy()))
    return np.array(insect_knowledges)

In [9]:
insect_knowledges = generate_insect_knowledge(model, all_dataloader)

In [10]:
for i in range(len(args.labels)):
    insect_filter = Y == i
    filtered_insect_knowledges = insect_knowledges[insect_filter]
    print("label = {}, knowledge = {}".format(i, np.round(filtered_insect_knowledges.mean(axis=0), decimals=3)))

label = 0, knowledge = [0.983 0.    0.008 0.007 0.001 0.001]
label = 1, knowledge = [0.    0.985 0.008 0.006 0.    0.   ]
label = 2, knowledge = [0.021 0.012 0.949 0.003 0.01  0.006]
label = 3, knowledge = [0.004 0.001 0.002 0.989 0.002 0.003]
label = 4, knowledge = [0.007 0.002 0.022 0.005 0.953 0.011]
label = 5, knowledge = [0.005 0.001 0.011 0.014 0.013 0.956]


In [11]:
with open(pj(args.knowledge_path, "knowledge.txt"), "w") as f:
    for i in range(len(args.labels)):
        insect_filter = Y == i
        filtered_insect_knowledges = insect_knowledges[insect_filter]
        filtered_insect_knowledge = filtered_insect_knowledges.mean(axis=0)
        knowledge_string = ""
        for j in range(len(args.labels)):
            if j == len(args.labels) - 1:
                knowledge_string += str(filtered_insect_knowledge[j]) + "\n"
            else:
                knowledge_string += str(filtered_insect_knowledge[j]) + " "
        f.write(knowledge_string)